
# Problem 1 - Hyperparameter Optimization using H20 20 points
In this question, you will compare the performances of H2O’s grid search and randomized grid search. You will use the H2ORandomForestEstimator model, and use the allyears2k headers.zip dataset used in the this link in the classification example.

1. Grid search

(a) Perform grid search for identifying the best hyperparameters for the H2ORandomForestEstimator model with ’ntrees’:[10,30,50,100] and ’max depth’: [1,2,4,6]. (2)

(b) Display the grid results, sorted by accuracy in a decreasing order. (2)

(c) Identify the best model and evaluate the model’s performance on a test set and display the AUC score.

In [1]:
! pip install h2o


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 177.5 MB 45 kB/s 
  Created wheel for h2o: filename=h2o-3.38.0.3-py2.py3-none-any.whl size=177541401 sha256=8b281128c879a01ce98a233e04905c6319be7f1f92d8e1a4edbe1b0883630f9f
  Stored in directory: /root/.cache/pip/wheels/16/f8/f4/69e1ff6a0d1cb61bdbc0d9888ee8437a1acf1eb1c6ffb8be20
Successfully built h2o


In [2]:
import h2o
from h2o.automl import H2OAutoML
h2o.init(nthreads = -1, max_mem_size = 8)
h2o.connect()

airlines= h2o.import_file("https://s3.amazonaws.com/h2o-public-test-data/smalldata/airlines/allyears2k_headers.zip")

# set the factors:
airlines["Year"]= airlines["Year"].asfactor()
airlines["Month"]= airlines["Month"].asfactor()
airlines["DayOfWeek"] = airlines["DayOfWeek"].asfactor()
airlines["Cancelled"] = airlines["Cancelled"].asfactor()
airlines['FlightNum'] = airlines['FlightNum'].asfactor()

# set the predictors and response columns:
predictors = ["Origin", "Dest", "Year", "UniqueCarrier",
              "DayOfWeek", "Month", "Distance", "FlightNum"]
response = "IsDepDelayed"


Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.17" 2022-10-18; OpenJDK Runtime Environment (build 11.0.17+8-post-Ubuntu-1ubuntu218.04); OpenJDK 64-Bit Server VM (build 11.0.17+8-post-Ubuntu-1ubuntu218.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.8/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp6zscx5fo
  JVM stdout: /tmp/tmp6zscx5fo/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp6zscx5fo/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.3
H2O_cluster_version_age:,8 days
H2O_cluster_name:,H2O_from_python_unknownUser_6krpm7
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,8 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


Connecting to H2O server at http://localhost:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.3
H2O_cluster_version_age:,8 days
H2O_cluster_name:,H2O_from_python_unknownUser_6krpm7
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,8 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [3]:
train, valid = airlines.split_frame(ratios = [.8], seed = 1234)


In [8]:

from h2o.estimators import H2ORandomForestEstimator
from h2o.grid.grid_search import H2OGridSearch

hyper_parameters = { 'ntrees':[10,30,50,100], 
                     'max_depth': [1,2,4,6] }

model_grid_search = H2OGridSearch(H2ORandomForestEstimator( nfolds=5),
      hyper_parameters, 
      grid_id="random_plus_manual")
    
model_grid_search.train(x=predictors,y=response, training_frame=train,validation_frame = valid)

drf Grid Build progress: |███████████████████████████████████████████████████████| (done) 100%


,max_depth,ntrees,model_ids,logloss
,6.0,100.0,random_plus_manual_model_16,0.6074097
,6.0,30.0,random_plus_manual_model_8,0.6080527
,6.0,50.0,random_plus_manual_model_12,0.6084735
,6.0,10.0,random_plus_manual_model_4,0.6113018
,4.0,50.0,random_plus_manual_model_11,0.6246213
,4.0,30.0,random_plus_manual_model_7,0.6250392
,4.0,100.0,random_plus_manual_model_15,0.6253382
,4.0,10.0,random_plus_manual_model_3,0.6254629
,2.0,100.0,random_plus_manual_model_14,0.6497448
,2.0,50.0,random_plus_manual_model_10,0.6497561


In [10]:
Rand_forest_model = model_grid_search.get_grid(sort_by='auc', decreasing=True)
Rand_forest_model

,max_depth,ntrees,model_ids,auc
,6.0,100.0,random_plus_manual_model_16,0.7356001
,6.0,50.0,random_plus_manual_model_12,0.7344319
,6.0,30.0,random_plus_manual_model_8,0.7330098
,6.0,10.0,random_plus_manual_model_4,0.7290180
,4.0,100.0,random_plus_manual_model_15,0.7199165
,4.0,30.0,random_plus_manual_model_7,0.7178729
,4.0,50.0,random_plus_manual_model_11,0.7172048
,4.0,10.0,random_plus_manual_model_3,0.7165073
,2.0,50.0,random_plus_manual_model_10,0.6919059
,2.0,100.0,random_plus_manual_model_14,0.6904609


In [11]:
best_model = Rand_forest_model.models[0]

# Now let's evaluate the model performance on a test set
# so we get an honest estimate of top model performance
best_model_perf1 = best_model.model_performance(valid)

best_model_perf1.auc()

0.7341018483113351


2. Randomized grid search

(a) Using the same model and hyperparameters grid, perform hyperparameter optimization using randomized grid search. Use a maximum of 10 models. (2)

(b) Display the results sorted by accuracy in a decreasing order. (2)

(c) Identify the best model and evaluate the model’s performance on a test set and display the auc score. (2)

In [13]:
search_criteria = { 'strategy': "RandomDiscrete", 'seed': 42,
                    'stopping_tolerance': 0.001,
                    'max_models': 10,
                   'max_runtime_secs': 120}
model_rand_search = H2OGridSearch(H2ORandomForestEstimator( nfolds=5),
      hyper_parameters, 
      grid_id="random_plus_manual",
      search_criteria=search_criteria)

model_rand_search.train(x=predictors,y=response, training_frame=train)

drf Grid Build progress: |███████████████████████████████████████████████████████| (done) 100%


,max_depth,ntrees,model_ids,logloss
,6.0,100.0,random_plus_manual_model_16,0.6074097
,6.0,30.0,random_plus_manual_model_8,0.6080527
,6.0,100.0,random_plus_manual_model_22,0.6082100
,6.0,50.0,random_plus_manual_model_12,0.6084735
,6.0,30.0,random_plus_manual_model_20,0.6085294
,6.0,50.0,random_plus_manual_model_28,0.6087198
,6.0,10.0,random_plus_manual_model_4,0.6113018
,6.0,10.0,random_plus_manual_model_23,0.6114164
,4.0,30.0,random_plus_manual_model_26,0.6244080
,4.0,50.0,random_plus_manual_model_11,0.6246213


In [16]:
Rand_forest_model_rand = model_rand_search.get_grid(sort_by='auc', decreasing=True)
Rand_forest_model_rand

,max_depth,ntrees,model_ids,auc
,6.0,100.0,random_plus_manual_model_16,0.7356001
,6.0,100.0,random_plus_manual_model_22,0.7345767
,6.0,50.0,random_plus_manual_model_12,0.7344319
,6.0,50.0,random_plus_manual_model_28,0.7336739
,6.0,30.0,random_plus_manual_model_20,0.7332505
,6.0,30.0,random_plus_manual_model_8,0.7330098
,6.0,10.0,random_plus_manual_model_4,0.7290180
,6.0,10.0,random_plus_manual_model_23,0.7283533
,4.0,100.0,random_plus_manual_model_15,0.7199165
,4.0,30.0,random_plus_manual_model_26,0.7187054


In [17]:
best_model = Rand_forest_model_rand.models[0]

# Now let's evaluate the model performance on a test set
# so we get an honest estimate of top model performance
best_model_perf1 = best_model.model_performance(valid)

best_model_perf1.auc()

0.7341018483113351

3. H2O AutoML

(a) Now using H20’s AutoML find the best deep learning model for the same classification task. Use H2OAutoML and test a maximum of 20 models to find the best performing model. (2)

(b) Display the leaderboard, and identify the best performing model using it and print its parameters. (2)

(c) Display the AUC score of the best model for the test set. (2)

(d) Identify the best XGBoost model among all the models tested ranked by log loss as the criteria. (2)